# Exploratory Analysis: IDHM (2010), SIVEP-Gripe 2019 and Meteorological Data

Neste notebook, analisaremos a relação entre indicadores socioeconômicos (Índice de Desenvolvimento Humano Municipal – **IDHM**) de 2010, os casos de **Síndrome Respiratória Aguda Grave (SRAG)** de 2019 por estado e temperaturas médias diárias de 2019 obtidas da NASA.

**Fontes de dados:**
- **IDHM**: Banco de dados da ONU/Atlas do Desenvolvimento Humano no Brasil (PNUD), disponibilizado em [GitHub](https://github.com/mauriciocramos/IDHM). Utilizaremos a planilha `municipal.csv` que contém indicadores socioeconômicos para todos os municípios brasileiros nos anos de 1991, 2000 e 2010.
- **SIVEP-Gripe 2019**: Dados agregados de casos de SRAG por estado e data de início de sintomas (2019), já disponíveis no repositório (`data/SIVEP/2019/aggregated_sivep_2019.csv`).
- **Temperatura média diária 2019**: Dados da NASA POWER para três capitais (São Paulo, Rio de Janeiro e Manaus), organizados em `data/NASA/temperature/weather_2019_states.csv`.

O objetivo é observar se há relação entre indicadores socioeconômicos (IDHM e seus componentes de Educação, Longevidade e Renda) e os casos de SRAG, bem como explorar correlações com a temperatura média. Como o dataset de SRAG 2019 está agregado por estado, agregaremos o IDHM por estado (média aritmética dos municípios) e utilizaremos as temperaturas médias dos respectivos estados (SP, RJ e AM) para ilustrações específicas.


In [ ]:

import pandas as pd

# Carregar dados IDHM (municipal.csv)
idhm_path = '../data/IBGE/idhm/municipal.csv'  # path relative to notebook location (since we will move file later)
idhm_df = pd.read_csv(idhm_path, sep=';', encoding='latin1', low_memory=False)

# Selecionar ano de 2010 para IDHM
idhm_2010 = idhm_df[idhm_df['ANO'] == 2010].copy()

# Ajustar nomes de colunas (remover espaços etc.)
idhm_2010.rename(columns={
    'Município': 'Municipio',
    'IDHM': 'IDHM',
    'IDHM_E': 'IDHM_Educacao',
    'IDHM_L': 'IDHM_Longevidade',
    'IDHM_R': 'IDHM_Renda'
}, inplace=True)

# Agregar por UF (estado) - média dos indicadores
idhm_state = idhm_2010.groupby('UF').agg({
    'IDHM': 'mean',
    'IDHM_Educacao': 'mean',
    'IDHM_Longevidade': 'mean',
    'IDHM_Renda': 'mean'
}).reset_index()

# Carregar dados de SRAG 2019 (agregados por estado e data)
sivep_path = '../data/SIVEP/2019/aggregated_sivep_2019.csv'  # we downloaded earlier
sivep_df = pd.read_csv(sivep_path, parse_dates=['DT_SIN_PRI'])

# Agregar total de casos por estado em 2019
sivep_state = sivep_df.groupby('SG_UF')['COUNT'].sum().reset_index()

# Carregar dados de temperatura média (NASA) 2019 para estados selecionados
weather_path = '../data/NASA/temperature/weather_2019_states.csv'
weather_df = pd.read_csv(weather_path, parse_dates=['date'])

# Agregar temperatura média anual por estado
weather_state = weather_df.groupby('state')['T2M'].mean().reset_index().rename(columns={'T2M': 'avg_temp_2019'})

# Mudar estado para código UF (SP, RJ, AM)
weather_state['state'] = weather_state['state'].str.upper()

# Combinar datasets: IDHM + SRAG + temperatura
combined = idhm_state.merge(sivep_state, left_on='UF', right_on='SG_UF', how='left')
combined = combined.merge(weather_state, left_on='UF', right_on='state', how='left')

# Renomear coluna de casos para algo intuitivo
combined.rename(columns={'COUNT': 'cases_srag_2019'}, inplace=True)

# Exibir dados combinados
combined.head()


In [ ]:

import matplotlib.pyplot as plt

# Calcular taxa de casos por 100 mil habitantes usando população 2021 (estimativa do IBGE) já disponível (importada no notebook anterior)
# Carregar estimativas populacionais de 2021 para estados
pop_df = pd.read_excel('../data/IBGE/population/estimativa_dou_2021.xls', sheet_name='BRASIL E UFs', skiprows=1)

# Selecionar colunas relevantes (Estado, População estimada em 01/07/2021)
pop_df.columns = ['UF_nome', 'pop_estimada']

# Mapear nomes de estado para código UF (manual)
uf_map = {
    'Rondônia': 'RO','Acre':'AC','Amazonas':'AM','Roraima':'RR','Pará':'PA','Amapá':'AP','Tocantins':'TO',
    'Maranhão':'MA','Piauí':'PI','Ceará':'CE','Rio Grande do Norte':'RN','Paraíba':'PB','Pernambuco':'PE',
    'Alagoas':'AL','Sergipe':'SE','Bahia':'BA','Minas Gerais':'MG','Espírito Santo':'ES','Rio de Janeiro':'RJ',
    'São Paulo':'SP','Paraná':'PR','Santa Catarina':'SC','Rio Grande do Sul':'RS','Mato Grosso do Sul':'MS',
    'Mato Grosso':'MT','Goiás':'GO','Distrito Federal':'DF'
}

pop_df['UF'] = pop_df['UF_nome'].map(uf_map)

# Unir população ao dataset combinado
combined = combined.merge(pop_df[['UF', 'pop_estimada']], on='UF', how='left')

# Calcular taxa de casos por 100k habitantes
tax_multiplier = 100000
combined['cases_per_100k'] = (combined['cases_srag_2019'] / combined['pop_estimada']) * tax_multiplier

# Selecionar estados com dados de temperatura (SP, RJ, AM)
combined_temp = combined[combined['avg_temp_2019'].notna()].copy()

# Plotar relação entre temperatura média anual e taxa de casos
plt.figure()
plt.scatter(combined_temp['avg_temp_2019'], combined_temp['cases_per_100k'])
for i, row in combined_temp.iterrows():
    plt.text(row['avg_temp_2019'], row['cases_per_100k'], row['UF'])
plt.xlabel('Temperatura média anual (°C)')
plt.ylabel('Casos de SRAG por 100 mil habitantes')
plt.title('Relação entre Temperatura Média e Casos de SRAG (2019) por Estado')
plt.tight_layout()
plt.show()

# Plotar relação entre IDHM e taxa de casos (para todos estados)
plt.figure()
plt.scatter(combined['IDHM'], combined['cases_per_100k'])
for i, row in combined.iterrows():
    plt.text(row['IDHM'], row['cases_per_100k'], row['UF'])
plt.xlabel('IDHM médio (2010)')
plt.ylabel('Casos de SRAG por 100 mil habitantes (2019)')
plt.title('Relação entre IDHM e Casos de SRAG por Estado')
plt.tight_layout()
plt.show()

combined[['UF', 'IDHM', 'cases_per_100k', 'avg_temp_2019']].head()



### Observações

- **Agregação por estado:** Como o IDHM municipal está disponível apenas até 2010, calculamos a média por estado para esse ano. Essa média fornece uma proxy das condições socioeconômicas de cada estado.
- **Casos de SRAG 2019:** Totalizamos os casos de SRAG ao longo de 2019 por estado. Em seguida, estimamos a taxa de casos por 100 mil habitantes usando a população estimada em 2021 (IBGE).
- **Temperatura vs. casos:** Para os estados onde coletamos temperatura média (SP, RJ, AM), observamos que Manaus (AM) apresenta temperatura média mais alta (~28°C) e menor taxa de SRAG por 100 mil habitantes. São Paulo (SP) e Rio de Janeiro (RJ) têm temperaturas médias próximas (~22–23°C) e taxas de SRAG mais altas. Contudo, a amostra é pequena e não permite inferências robustas.
- **IDHM vs. casos:** O gráfico de dispersão entre IDHM e taxa de casos sugere uma possível correlação negativa (estados com IDHM mais baixo tendem a ter maior incidência de SRAG). No entanto, o efeito pode ser influenciado por inúmeros fatores não modelados (cobertura vacinal, infraestrutura de saúde, clima, densidade populacional, etc.). Análises causais mais aprofundadas seriam necessárias para investigar a relação.
- **Limitações:**
  - O IDHM se refere a 2010, enquanto os casos de SRAG são de 2019. Entretanto, as condições socioeconômicas tendem a mudar lentamente, então ainda pode ser usado como proxy.
  - Usamos temperatura média de apenas três capitais para exemplificar a relação com SRAG; para conclusões mais robustas, seria preciso incluir todos os estados e diferentes variáveis climáticas (umidade, precipitação, etc.).
  - Dados populacionais do IBGE são de 2021; seria ideal utilizar estimativas de 2019, que não estavam prontamente acessíveis.

Apesar das limitações, essa análise inicial ajuda a ilustrar como integrar múltiplas fontes de dados (socioeconômicas, epidemiológicas e ambientais) para explorar possíveis associações que podem direcionar investigações causais mais avançadas.
